# PRIMERA PARTE
### En esta seccion decidi crear un codigo que creara dos carpetas (Tumor y No_tumor) cada una con las respectivas imagenes ya separadas (esto lo hice en base al csv donde ya sabemos a que clase pertenece cada imagen), esto lo hice asi ya que fue la forma mas eficaz que encontre para crear la CNN (https://www.youtube.com/watch?v=j-3vuBynnOE&list=PLQVvvaa0QuDfhTox0AjmQ6tvTgMBZBEXN&index=2). Todo este codigo lo hice en Pycharm ya que fue el unico entorno donde pude utilizar todas las librerias sin problema.  

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
###PRIMERA PARTE DEL CODIGO

df = pd.read_csv(r"C:\Users\vladi\Downloads\segundo parcial datos\Brain Tumor.csv")

df_01 = df[df['Class'] == 1]
df_00 = df[df['Class'] == 0]

# Nombres de imágenes
tumor = list(df_01["Image"])
no_tumor = list(df_00["Image"])

# Carpetas de origen y destino
carpeta_origen = r"C:\Users\vladi\Videos\Dataset_BrainTumor"
carpeta_destino_tumor = r"C:\Users\vladi\Videos\Brain_tumor\Tumor"
carpeta_destino_no_tumor = r"C:\Users\vladi\Videos\Brain_tumor\No_tumor"

# Aqui hacemos las carpetas (usamos el exist para que las reconozca)
os.makedirs(carpeta_destino_tumor, exist_ok=True)
os.makedirs(carpeta_destino_no_tumor, exist_ok=True)
# Extensión de las imágenes
ext = ".jpg"

# la función para copiar imágenes
def copiar_imagenes(lista, carpeta_destino):
    for nombre in lista:
        nombre_archivo = nombre + ext
        ruta_origen = os.path.join(carpeta_origen, nombre_archivo)
        ruta_destino = os.path.join(carpeta_destino, nombre_archivo)
        if os.path.exists(ruta_origen):
            shutil.copyfile(ruta_origen, ruta_destino)
        else:
            print(f"No encontrada: {ruta_origen}")

# Copiar las imágenes clasificadas
copiar_imagenes(tumor, carpeta_destino_tumor)
copiar_imagenes(no_tumor, carpeta_destino_no_tumor)

### aqui ya tenemos dos carpetas cada una con las imagenes ya separadas. Ahora toca crear la CNN y compararla con las otras tecnicas



# SEGUNDA PARTE
### Aqui ya he hecho la CNN que nos separa las imagenes y comparamos los resultados con los otros metodos. 

In [ ]:
DATADIR = r"C:\\Users\\vladi\\Videos\\Brain_tumor"
CATEGORIES = ["Tumor", "No_tumor"]
IMG_SIZE = 100

# Cargar imágenes y etiquetas
X, y = [], []
for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    label = CATEGORIES.index(category)
    for img_name in os.listdir(path):
        try:
            img_path = os.path.join(path, img_name)
            img = Image.open(img_path).convert("L")
            img = img.resize((IMG_SIZE, IMG_SIZE))
            X.append(np.array(img))
            y.append(label)
        except Exception:
            pass

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) / 255.0
y = np.array(y)

# Separar en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=5, validation_split=0.1, verbose=2)

# Predicciones del modelo CNN
cnn_preds = np.argmax(model.predict(X_test), axis=1)

# Clasificación Gaussiana
X_flat = X.reshape(len(X), -1)
X_train_flat, X_test_flat = train_test_split(X_flat, test_size=0.2, random_state=42)

gnb = GaussianNB()
gnb.fit(X_train_flat, y_train)
gnb_preds = gnb.predict(X_test_flat)

# Análisis Discriminante Lineal
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_flat, y_train)
lda_preds = lda.predict(X_test_flat)

# Matrices de confusión
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
titles = ["CNN", "Gaussian NB", "LDA"]
preds = [cnn_preds, gnb_preds, lda_preds]

for i in range(3):
    cm = confusion_matrix(y_test, preds[i])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=CATEGORIES)
    disp.plot(ax=ax[i], cmap='GnBu', colorbar=False)
    ax[i].set_title(titles[i])

plt.tight_layout()
plt.show()

![imagen](C:\Users\vladi\Videos\Matriz_confusion_cnn.png)